In [1]:
import sys
sys.path.insert(1,"../")
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import SimpleITK as sitk
import random
from sklearn.model_selection import train_test_split
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.profilers import SimpleProfiler
from lightning.pytorch.callbacks import ModelCheckpoint
import torcheval.metrics as tm
import os
from models.mwcnn.mwcnn_model import MWCNN
from src.features.build_features import *

/home/mohamed/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
random.seed(10)

In [3]:
data = (pd.read_csv("../scratch_pad/csv_files/new_merged.csv")).drop(columns='index')

In [4]:
train , test = train_test_split(data,test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [5]:
train, val = train_test_split(train, test_size=0.1)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [6]:
class dataset(Dataset):
    def __init__(self,data_frame,image_compose,mask_compoe):
        self.data_frame = data_frame
        self.image_compose = image_compose
        self.mask_compoe = mask_compoe

    def __getitem__(self,item):
        file = self.data_frame.iloc[item]['filename']
        if self.image_compose:
            image = self.image_compose(file)
        if self.mask_compoe:
            mask = self.mask_compoe(file)
        return image,mask
    def __len__(self):
        return len(self.data_frame)


In [7]:
train_transformation = transforms.Compose(
    [
        transforms.Lambda(convert_to_pil),
        transforms.Lambda(to_numpy),
        transforms.ToTensor(),
        transforms.Lambda(wrapper_function),
        transforms.Resize((768,768)),
        transforms.Lambda(normalize),
    ]
)
val_transformation = transforms.Compose(
    [
        transforms.Lambda(convert_to_pil),
        transforms.Lambda(to_numpy),
        transforms.ToTensor(),
        transforms.Lambda(wrapper_function),
        transforms.Resize((768,768)),
        transforms.Lambda(normalize),
    ]
)

mask_transformation = transforms.Compose(
    [
        transforms.Lambda(convert_to_pil),
        transforms.Lambda(to_numpy),
        transforms.ToTensor(),
        transforms.Resize((768,768)),
        transforms.ConvertImageDtype(dtype=torch.float32),
    ]
)


In [8]:
ds_train = dataset(train,train_transformation,mask_transformation)

In [9]:
ds_val = dataset(val,val_transformation,mask_transformation)

In [10]:
dl_train = DataLoader(ds_train, batch_size=1, shuffle=True)
dl_val = DataLoader(ds_val, batch_size=1, shuffle=False)

In [11]:
class LightningModel(pl.LightningModule):
    def __init__(self, metric,model,optimizer,loss_fn,log_lr):
        super().__init__()
        self.model = model
        self.metric = metric
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.log_lr = log_lr
        self.n_steps_train=0
        self.loss_per_epoch_train=0
        self.n_steps_val=0
        self.loss_per_epoch_val=0
        self.current_val_epoch=0
        

    def on_train_epoch_start(self):
        self.n_steps_train=0
        self.loss_per_epoch_train=0
        if self.log_lr:
            self.log_dict({"learning_rate":self.configure_optimizers()[0].state_dict()['param_groups'][0]["lr"]},logger=True)
    

    def training_step(self, batch, batch_idx):
        inputs, target = batch
        inputs, target = inputs.to('cpu'), target.to('cpu')
        output = self.model(inputs)
        train_loss = self.loss_fn(output, target)
        scores = self.metric
        scores.update(output,target)
        metric = scores.compute().item()
        self.log_dict({"train_loss":metric},on_epoch=True,on_step=True,prog_bar=True,logger=True)
        self.n_steps_train+=1
        self.loss_per_epoch_train+=train_loss
        return train_loss

    def on_training_epoch_end(self, outs):
        avg=float(self.loss_per_epoch_train)/float(self.n_steps_train)
        self.log_dict({"train_loss_avg":avg},on_epoch=True,prog_bar=True,logger=True)

    def configure_optimizers(self):
        return [self.optimizer]

In [12]:
model = MWCNN(1,1).to('cpu')
loss_fn = torch.nn.MSELoss()
metric = tm.PeakSignalNoiseRatio(device='cpu')
optimizer=torch.optim.AdamW(model.parameters(),lr=0.003,weight_decay=5e-3)
lightning_module=LightningModel(metric=metric,model=model,optimizer=optimizer,loss_fn=loss_fn,log_lr=True)

callbacks_compositions=[
                        ModelCheckpoint(dirpath="../scratch_pad/model_checkpoints",
                                        monitor="val_loss_avg",
                                        filename="MWCNN_TEST"+'-{epoch}-{val_loss_avg:.2f}',
                                        save_top_k=5,
                                        save_weights_only=False)]
logger = TensorBoardLogger(save_dir="../scratch_pad/tesnorboard_logs", version=1, name="MWCNN_TEST")
profiler = SimpleProfiler(dirpath=os.getcwd(),filename="MWCNN_TEST")


In [13]:
trainer = pl.Trainer(
            max_epochs=2,
            profiler=profiler,
            logger=logger,
            accelerator='cpu',
            # devices=1,
            # accumulate_grad_batches=10,
            callbacks=callbacks_compositions,
            log_every_n_steps=1,
            # precision=32,
            # limit_train_batches=100,
            # limit_val_batches=100,
            # auto_lr_find=True,
            )
trainer.fit(model=lightning_module, train_dataloaders=dl_train)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/mohamed/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
2024-01-02 15:22:48.647237: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 15:22:48.752477: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-02 15:22:49.

Training: |          | 0/? [00:00<?, ?it/s]

x2 -1 -1
<built-in method get_device of Tensor object at 0x7f2629495b80>
x2 -1 -1
<built-in method get_device of Tensor object at 0x7f26280adf90>
x2 -1 -1


/home/mohamed/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
